In [2]:

tokenizer=AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model=AutoModelForCausalLM.from_pretrained("google/gemma-2-9b-it", device_map="auto")

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.62s/it]


In [5]:
# model2=AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map="auto")
# tokenizer2=AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards: 100%|██████████| 3/3 [00:51<00:00, 17.25s/it]


In [4]:
model2=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", device_map="auto")
tokenizer2=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.89s/it]


In [4]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", token=HF_TOKEN)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.79s/it]


In [32]:
# Load GSM8K dataset
dataset = load_dataset("openai/gsm8k", 'main')
train_data = list(dataset["train"])

subset_size = 750
test_data = list(dataset["test"])[:subset_size]

In [41]:
BATCH_SIZE = 4

import csv
output_csv_path = f"/home/[username]/ondemand/data/sys/myjobs/projects/CSE576/{subset_size}_predictions"

# Evaluation function with batching
def evaluate_model(test_data, train_data, n_shot, generator, batch_size=BATCH_SIZE):
    total = correct = 0
    log_file_path = 'log/errors.txt'
    os.makedirs(os.path.dirname(log_file_path), exist_ok=True)

    # Clear log file
    with open(log_file_path, 'w') as log_file:
        log_file.write('')
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['question', 'reasoning', 'pred', 'actual'])

    # Process test data in batches
    for i in tqdm(range(0, len(test_data), batch_size), desc="Evaluating"):
        batch_data = test_data[i:i + batch_size]
        
        # Generate prompts for the entire batch
        batch_prompts = []
        for qna in batch_data:
            messages = nshot_chats(nshot_data=train_data, n=n_shot, question=qna['question'])
            prompt_text = "\n".join([f"{m['content']}\n" for m in messages])
            batch_prompts.append(prompt_text)

        # Generate responses for the batch
        with torch.no_grad():
            responses = generator(batch_prompts, max_new_tokens=512, do_sample=False, temperature=0.0)

        # Process each response in the batch
        for idx, response in enumerate(responses):
            prompt = batch_prompts[idx]
            generated_text = response[0]['generated_text'].replace(prompt, "", 1)
            qna = batch_data[idx]  # Corresponding question-answer pair
            
            # Extract predicted answer and ground truth answer
            pred_reas, pred_ans = extract_ans_from_response(generated_text)
            #print(f"\n\n{pred_reas}\n\n{pred_ans}\n\n")
            true_reas, true_ans = extract_ans_from_response(qna['answer'])

            with open(output_csv_path, 'a', newline='', encoding='utf-8') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow([qna['question'], pred_reas, pred_ans, true_ans])

            # Accuracy calculation
            total += 1
            if pred_ans != true_ans:
                # Log incorrect predictions
                with open(log_file_path, 'a', encoding='utf-8') as log_file:
                    log_file.write(f"Prompt:\n{batch_prompts[idx]}\n\n")
                    log_file.write(f"Response: {generated_text}\n\n")
                    log_file.write(f"Ground Truth: {qna['answer']}\n\n")
                    log_file.write(f"Current Accuracy: {correct/total:.3f}\n\n")
                    log_file.write('\n\n')
                print(f"incorrect: prediction: {pred_ans}\tactual: {true_ans}")
            else:
                correct += 1
                #print("correct")

    # Final accuracy
    accuracy = correct / total
    print(f"Total Accuracy: {accuracy:.3f}")
    return accuracy

In [42]:
# Run the evaluation
accuracy = evaluate_model(test_data, train_data, N_SHOT, generator)
print(f"Final Accuracy: {accuracy:.3f}")

Evaluating:   1%|          | 1/188 [01:10<3:40:36, 70.78s/it]

incorrect: prediction: 65000	actual: 70000


Evaluating:   1%|          | 2/188 [02:21<3:39:57, 70.95s/it]

Marker '####' not found in response.
incorrect: prediction: None	actual: 160


Evaluating:   2%|▏         | 3/188 [03:32<3:38:59, 71.03s/it]

...

incorrect: prediction: 30	actual: 18


Evaluating:  98%|█████████▊| 184/188 [3:37:42<04:43, 70.97s/it]

incorrect: prediction: 19	actual: 17


Evaluating: 100%|██████████| 188/188 [3:41:51<00:00, 70.80s/it]

incorrect: prediction: 6	actual: 3
Total Accuracy: 0.812
Final Accuracy: 0.812
